# Tarea 3: Regularización y Optimización <br/> CC6204 Deep Learning, Universidad de Chile  <br/> Hoja de respuestas
## Nombre: José Manuel Rubio

**Fecha de entrega: 13 de noviembre de 2020**


In [ ]:
# Este notebook está pensado para correr en CoLaboratory. 
# Lo único imprescindible por importar es torch
import torch

# Posiblemenete quieras instalar e importar ipdb para debuggear.
# Si es así, descomenta lo siguiente:
# !pip install -q ipdb
# import ipdb

# Aqui instalamos la libreria de correccion del curso
!pip install -U "git+https://github.com/dccuchile/CC6204.git@master#egg=cc6204&subdirectory=autocorrect"
from timeit import default_timer as timer

  Cloning https://github.com/dccuchile/CC6204.git (to revision master) to /tmp/pip-install-ir6k_7h5/cc6204
  Running command git clone -q https://github.com/dccuchile/CC6204.git /tmp/pip-install-ir6k_7h5/cc6204
  Created wheel for cc6204: filename=cc6204-0.5.0-cp36-none-any.whl size=5801 sha256=320e200b2dc92dd5789f73fa26d01283329cf1ceb367c4110859b452515896be
  Stored in directory: /tmp/pip-ephem-wheel-cache-yc_5t9_3/wheels/62/f0/30/aadcb7ce24a2f9c935890518e902d4e23bf97b80f47bb64414
Successfully built cc6204


In [ ]:
# importamos las herramientas del curso
from cc6204 import AutoCorrect, FailedTest

# ingresa el host y port que posteamos en u-cursos
corrector = AutoCorrect(host="cc6204.dcc.uchile.cl", port=443)

# anota el token que te daremos en u-cursos
token = "]ye/Ox;nsz"

Connection stablished


# Parte 1: Regularización y Generalización



## 1a) Regularización por *weight decay*

In [ ]:
# Tu código debiera continuar así
class SGD():
  def __init__(self, parameters, lr, beta=0):
    # lo que sea necesario inicializar
    self.parameters = parameters
    self.lr = lr
    self.beta = beta
  
  def step(self):
    # actualiza acá los parámetros a partir de los gradientes
    # y considera el nuevo valor beta
    for p in self.parameters:
      p.data = (1 - self.beta)*p.data - self.lr * p.grad

In [ ]:
# Tests del API del curso
weight, grad = corrector.get_test_data(homework=3, question="1a", test=1, token=token)

weight = torch.tensor(weight, requires_grad=True)
weight.grad = torch.tensor(grad)

optimizer = SGD([weight], lr=0.1, beta=0.1)
optimizer.step()

# Submit
corrector.submit(homework=3, question="1a", test=1, token=token, answer=weight)

Correct Test!


## 1b) Regularización por dropout

Para esta parte de la tarea, va a ser necesario modificar el método `forward` para que entregue el valor a la salida de la i-esima capa escondida. Para esto se modifica el método forward para que reciba un parámetro `output_layer` que indica luego de que capa escondida se espera el output.

In [ ]:
# Tu código debiera continuar como sigue

class FFNN():
  def __init__(self, F, l_h, l_a, C, keep_prob=None):
    # debes crear los parámetros necesarios para hacer 
    # dropout en cada capa dependiendo de keep_prob
    super(FFNN, self).__init__()
    
    sizes = [F] + l_h + [C]
    self.Ws = torch.nn.ParameterList([torch.nn.Parameter(torch.randn(sizes[i], sizes[i+1])) for i in range(len(sizes)-1)])
    self.bs = torch.nn.ParameterList([torch.nn.Parameter(torch.zeros(h)) for h in sizes[1:]])
    self.fs = l_a
    self.l_h = l_h
    self.keep_prob = keep_prob
  
  def forward(self, x, predict=False, output_layer=None):
    # debes modificar esta función para considerar el dropout
    # y preocuparte de no considerarlo cuando (predict=True)
    u_layers = []
    h_layers = []
    for W, b, f, n_h, prob in zip(self.Ws[:-1], self.bs[:-1], self.fs, self.l_h, self.keep_prob):
      if predict == True:
        u = torch.mm(x, W) + b
        x = f(u)
      else:
        neurons_hi, prob_on = n_h, prob
        prob_neurons = torch.rand(neurons_hi)
        cols = torch.range(0, neurons_hi - 1, dtype=torch.float32)
        ctk = torch.where(prob_neurons <= prob_on, cols, prob_neurons).type(torch.int64)      # Colmuns to Keep
        ctk = ctk[ctk.nonzero().detach()]
        ctk = torch.reshape(ctk, (len(ctk),))
        W_ctk = W.index_select(dim=1, index=ctk)
        b_ctk = b.index_select(dim=0, index=ctk)
        u = torch.mm(x, W_ctk) + b_ctk
        x = f(u)
      u_layers.append(u)
      h_layers.append(x)

    self.U_layers = u_layers
    self.H_layers = h_layers
    soft = torch.nn.Softmax(dim=1)
    if output_layer != None:
      return h_layers[output_layer]
    else:
      return soft(torch.mm(x, self.Ws[-1]) + self.bs[-1])
  
  def backward(self, x, y, y_pred):
    # computar acá todos los gradientes considerando el dropout 
    # Ultima capa:
    q_hlayer = len(self.fs)
    B = x.size()[0]
    dL_duL1 = (y_pred - y)/B
    dL_duL = self.H_layers[-1].t() @ dL_duL1
    dL_dc = torch.sum(dL_duL1, dim=0)
    dL_dhk = dL_duL1 @ self.Ws[-1].t()
    grads, params = [], []
    functions_inv, W_inv, h_inv = list(reversed(self.fs)), list(reversed(self.Ws[:-1])), list(reversed(self.H_layers)) # Volteando la lista con los parametros
    U_inv, b_inv = list(reversed(self.U_layers)), list(reversed(self.bs[:-1]))
    # Iteración para las capas ocultas
    for k in range(q_hlayer):
      f_k, u_k, w_k = functions_inv[k], U_inv[k], W_inv[k]
      dL_duk = dL_dhk * f_k(u_k, gradient=True)
      dL_dbk = torch.sum(dL_duk, dim=0)
      dL_dhk1 = dL_duk @ w_k.t()
      dL_dwk = h_inv[k+1].t() @ dL_duk if not k == q_hlayer - 1 else x.t() @ dL_duk
      dL_dhk = dL_dhk1
      grads.append(dL_dwk)
      params.append(w_k)
      grads.append(dL_dbk)
      params.append(b_inv[k])
    grads.reverse()
    grads.append(dL_duL)
    grads.append(dL_dc)
    params.reverse()
    params.append(self.Ws[-1]) # Se añade U de la ultima capa
    params.append(self.bs[-1]) # Se añade C de la última capa
    Grads, Params = grads[:], params[:]
    for p, g in zip(Params, Grads):
      p.grad = g   

In [ ]:
def sig(T):
  return torch.reciprocal(1 + torch.exp(-1 * T))

In [ ]:
# Tests del API del curso
torch.manual_seed(0)
sample = torch.rand(1, 10)
red = FFNN(10, [1000], [sig], 1, keep_prob=[1.0, 0.5])
y = red.forward(sample, output_layer=0)
output_mask = (y == 0)
percent = torch.sum(output_mask).item() / list(output_mask.size())[-1]

# Submit
corrector.submit(homework=3, question="1b", test=1, token=token, answer=percent)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)


Correct Test!


## 1c) Entrenamiento y generalización sobre MNIST 

In [ ]:
# Aqui el codigo para entrenar en MNIST
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

mnist_train = MNIST('mnist', train=True, transform=ToTensor(), download=True)
mnist_test = MNIST('mnist', train=False, transform=ToTensor(), download=True)

train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=True)

In [ ]:
len(dataset)

60000

In [ ]:
dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

# Parte 2: Optimización

## 2a) Inicialización de Xavier/He

Para los test de esta parte vamos a necesitar que modifiques tu código para que se pueda entregar valores predeterminados de `r`. Ahora tu código para las inicializaciones debe ser: `xavier_init(first_dim, second_dim, r=None)`, `he_init(first_dim, second_dim, r=None)`.

In [ ]:
# Tu código debiera continuar como sigue
def xavier_init(first_dim, second_dim, r=None):
    return r*torch.sqrt(torch.tensor(1/first_dim))

def he_init(first_dim, second_dim, r=None):
    return r*torch.sqrt(torch.tensor(2/first_dim))

In [ ]:
torch.sqrt(torch.tensor(1/4))

tensor(0.5000)

In [ ]:
# Tests del API del curso
r_xavier = corrector.get_test_data(homework=3, question="2a", test=1, token=token)
r_he = corrector.get_test_data(homework=3, question="2a", test=2, token=token)

w_xavier = xavier_init(50, 50, torch.tensor(r_xavier))
w_he = he_init(50, 50, torch.tensor(r_he))

corrector.submit(homework=3, question="2a", test=1, token=token, answer=w_xavier)
corrector.submit(homework=3, question="2a", test=2, token=token, answer=w_he)

Correct Test!
Correct Test!


## 2b) Descenso de gradiente con momentum

In [ ]:
# Tu código debiera continuar así

class SGD():
  def __init__(self, parameters, lr, momentum=0):
    # lo que sea necesario inicializar
    self.parameters = parameters
    self.lr = lr
    self.momentum = momentum
    self.V_theta = [torch.zeros(p.data.size()) for p in parameters]
  
  def step(self):
    # actualiza acá los parámetros a partir de los gradientes
    # y considerando el valor de momentum que acabámos de agregar
    for p, v in zip(self.parameters, self.V_theta):
      v.data = self.momentum*v.data - self.lr*p.grad
      p.data += v.data

In [ ]:
# Tests del API del curso
weight, grad = corrector.get_test_data(homework=3, question="2b", test=1, token=token)

weight = torch.tensor(weight, requires_grad=True)
weight.grad = torch.tensor(grad)

optimizer = SGD([weight], lr=0.1, momentum=0.9)
optimizer.step()

# Submit
corrector.submit(homework=3, question="2b", test=1, token=token, answer=weight)
optimizer.step()
corrector.submit(homework=3, question="2b", test=2, token=token, answer=weight)

Correct Test!
Correct Test!


## 2c) RMSProp

In [ ]:
# Tu código acá

class RMSProp():
  def __init__(self, red, lr=0.001, beta=0.9, epsilon=1e-8):
    # en este caso debes inicializar la variable que acumula
    # el promedio exponencial de los cuadrados
    self.parameters = red
    self.beta = beta
    self.epsilon = epsilon
    self.lr = lr
    self.S_theta = [torch.zeros(p.data.size()) for p in red]
  
  def step(self):
    # actualiza acá los parámetros a partir de los gradientes
    # y la corrección según S
    s_data, p_data = [], []
    for p, s in zip(self.parameters, self.S_theta):
      s = self.beta*s + (1 - self.beta)*(p.grad*p.grad)
      p.data -= self.lr/(torch.sqrt(s) + torch.tensor([self.epsilon]))*p.grad
      s_data.append(s)
      p_data.append(p)
    self.S_theta, self.parameters = s_data, p_data 

In [ ]:
# Tests del API del curso
weight, grad = corrector.get_test_data(homework=3, question="2c", test=1, token=token)

weight = torch.tensor(weight, requires_grad=True)
weight.grad = torch.tensor(grad)

optimizer = RMSProp([weight], lr=0.001, beta=0.9, epsilon=1e-8)
optimizer.step()

# Submit
corrector.submit(homework=3, question="2c", test=1, token=token, answer=weight)
optimizer.step()
corrector.submit(homework=3, question="2c", test=2, token=token, answer=weight)

Correct Test!
Correct Test!


## 2d) Adam

In [ ]:
# Tu código acá

class Adam():
  def __init__(self, red, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    # en este caso debes inicializar la variable que acumula
    # el promedio exponencial de los cuadrados
    n = 1
    self.red = red
    self.beta1 = beta1
    self.beta2 = beta2
    self.epsilon = epsilon
    self.lr = lr
    self.S_theta = [torch.zeros(p.data.size()) for p in red]
    self.P_theta = [torch.zeros(p.data.size()) for p in red]
    self.n = n

  def step(self):
    # actualiza acá los parámetros a partir de los gradientes
    # y la corrección según S
    Param_data, P_data, S_data,  = [], [], []
    for param, s, p in zip(self.red, self.S_theta, self.P_theta):
      p.data = self.beta1*p.data + (1 - self.beta1)*param.grad
      s.data = self.beta2*s.data + (1 - self.beta2)*(param.grad * param.grad)
      p_hat, s_hat = (p.data)/(1-self.beta1**(self.n)), (s.data)/(1-self.beta2**(self.n))
      param.data -= (self.lr)/(torch.sqrt(s_hat) + self.epsilon)*p_hat
      Param_data.append(param)
      P_data.append(p)
      S_data.append(s)
    self.n += 1
    self.red, self.S_theta, self.P_theta = Param_data, S_data, P_data

In [ ]:
# Tests del API del curso
weight, grad = corrector.get_test_data(homework=3, question="2d", test=1, token=token)

weight = torch.tensor(weight, requires_grad=True)
weight.grad = torch.tensor(grad)

optimizer = Adam([weight], lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8)
optimizer.step()

# Submit
corrector.submit(homework=3, question="2d", test=1, token=token, answer=weight)
optimizer.step()
corrector.submit_check_some(homework=3, question="2d", tests=[2, 3], token=token,
                            answer_dict={2: weight, 3: weight}, required_number=1)

Correct Test!
Correct Test!


## 2e) Entrenamiento en MNIST 

Usa tu red neuronal para entrenar con los datos de MNIST y compara cómo cambian las curvas de entrenamiento dependiendo de factores como la inicialización y los algoritmos que utilices. Presenta al menos dos gráficos en donde compares. Por ejemplo, puedes presentar uno que para la misma estrategia de inicialización, los tres algoritmos de optimización para varias épocas y cómo evoluciona la pérdida y el acierto. En cada caso comenta que conclusiones puedes sacar. Algunos ejemplos de preguntas que podrías tratar de responder son:
* ¿cómo afecta el algoritmo de optimización al tiempo de convergencia de la red para los datos de entrenamiento?
* ¿cómo afecta el algoritmo de optimización en el acierto alcanzado por la red en los datos de prueba?
* Si haces la parte opcional de Batch Normalization, puedes también preguntarte cosas como si aplicar, o no, BN afecta a todos los algoritmos de optimización por igual.

In [ ]:
# Aqui el codigo para entrenar en MNIST

# Parte 3 (Opcional): Batch Normalization

In [ ]:
# Tu código debiera continuar como sigue

class FFNN():
  def __init__(self, F, l_h, l_a, C, keep_prob=None, bn=None):
    # debes crear los parámetros necesarios para las capas de
    # batch normalizacion
    pass
  
  def forward(x, predict=False):
    # debes modificar esta función para considerar las capas para las que se
    # usará batch normalization
    # también debes preocuparte de guardar los datos estadísticos que se
    # usaran en tiempo de test (predict=True)
    pass
  
  def backward(x,y,y_pred):
    # computar acá todos los gradientes considerando las capas de 
    # batch normalization
    # no olvides considerar los nuevos parámetros entrenables.
    pass